# Optional Lab - Softmax Function
In this lab, we will explore the softmax function. This function is used in both Softmax Regression and in Neural Networks when solving Multiclass Classification problems.  

<center>  <img  src="./images/C2_W2_Softmax_Header.PNG" width="600" />  <center/>

  

In [3]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from IPython.display import display, Markdown, Latex
from sklearn.datasets import make_blobs
%matplotlib widget
from matplotlib.widgets import Slider
from lab_utils_common import dlc
from lab_utils_softmax import plt_softmax
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

> **Note**: Normally, in this course, the notebooks use the convention of starting counts with 0 and ending with N-1,  $\sum_{i=0}^{N-1}$, while lectures start with 1 and end with N,  $\sum_{i=1}^{N}$. This is because code will typically start iteration with 0 while in lecture, counting 1 to N leads to cleaner, more succinct equations. This notebook has more equations than is typical for a lab and thus  will break with the convention and will count 1 to N.

## Softmax Function
In both softmax regression and neural networks with Softmax outputs, N outputs are generated and one output is selected as the predicted category. In both cases a vector $\mathbf{z}$ is generated by a linear function which is applied to a softmax function. The softmax function converts $\mathbf{z}$  into a probability distribution as described below. After applying softmax, each output will be between 0 and 1 and the outputs will add to 1, so that they can be interpreted as probabilities. The larger inputs  will correspond to larger output probabilities.
<center>  <img  src="./images/C2_W2_SoftmaxReg_NN.png" width="600" />  

The softmax function can be written:
$$a_j = \frac{e^{z_j}}{ \sum_{k=1}^{N}{e^{z_k} }} \tag{1}$$
The output $\mathbf{a}$ is a vector of length N, so for softmax regression, you could also write:
\begin{align}
\mathbf{a}(x) =
\begin{bmatrix}
P(y = 1 | \mathbf{x}; \mathbf{w},b) \\
\vdots \\
P(y = N | \mathbf{x}; \mathbf{w},b)
\end{bmatrix}
=
\frac{1}{ \sum_{k=1}^{N}{e^{z_k} }}
\begin{bmatrix}
e^{z_1} \\
\vdots \\
e^{z_{N}} \\
\end{bmatrix} \tag{2}
\end{align}


Which shows the output is a vector of probabilities. The first entry is the probability the input is the first category given the input $\mathbf{x}$ and parameters $\mathbf{w}$ and $\mathbf{b}$.  
Let's create a NumPy implementation:

In [4]:
def my_softmax(z):
    ez = np.exp(z)              #element-wise exponenial
    sm = ez/np.sum(ez)
    return(sm)

Below, vary the values of the `z` inputs using the sliders.

In [6]:
plt.close("all")
plt_softmax(my_softmax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

As you are varying the values of the z's above, there are a few things to note:
* the exponential in the numerator of the softmax magnifies small differences in the values 
* the output values sum to one
* the softmax spans all of the outputs. A change in `z0` for example will change the values of `a0`-`a3`. Compare this to other activations such as ReLU or Sigmoid which have a single input and single output.

## Cost
<center> <img  src="./images/C2_W2_SoftMaxCost.png" width="400" />    <center/>

The loss function associated with Softmax, the cross-entropy loss, is:
\begin{equation}
  L(\mathbf{a},y)=\begin{cases}
    -log(a_1), & \text{if $y=1$}.\\
        &\vdots\\
     -log(a_N), & \text{if $y=N$}
  \end{cases} \tag{3}
\end{equation}

Where y is the target category for this example and $\mathbf{a}$ is the output of a softmax function. In particular, the values in $\mathbf{a}$ are probabilities that sum to one.
>**Recall:** In this course, Loss is for one example while Cost covers all examples. 
 
 
Note in (3) above, only the line that corresponds to the target contributes to the loss, other lines are zero. To write the cost equation we need an 'indicator function' that will be 1 when the index matches the target and zero otherwise. 
    $$\mathbf{1}\{y == n\} = =\begin{cases}
    1, & \text{if $y==n$}.\\
    0, & \text{otherwise}.
  \end{cases}$$
Now the cost is:
\begin{align}
J(\mathbf{w},b) = -\frac{1}{m} \left[ \sum_{i=1}^{m} \sum_{j=1}^{N}  1\left\{y^{(i)} == j\right\} \log \frac{e^{z^{(i)}_j}}{\sum_{k=1}^N e^{z^{(i)}_k} }\right] \tag{4}
\end{align}

Where $m$ is the number of examples, $N$ is the number of outputs. This is the average of all the losses.


## Tensorflow
This lab will discuss two ways of implementing the softmax, cross-entropy loss in Tensorflow, the 'obvious' method and the 'preferred' method. The former is the most straightforward while the latter is more numerically stable.

Let's start by creating a dataset to train a multiclass classification model.

In [7]:
# make  dataset for example
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
X_train, y_train = make_blobs(n_samples=2000, centers=centers, cluster_std=1.0,random_state=30)

### The *Obvious* organization

The model below is implemented with the softmax as an activation in the final Dense layer.
The loss function is separately specified in the `compile` directive. 

The loss function is `SparseCategoricalCrossentropy`. This loss is described in (3) above. In this model, the softmax takes place in the last layer. The loss function takes in the softmax output which is a vector of probabilities. 

In [8]:
model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'softmax')    # < softmax activation here
    ]
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 1s 1ms/step - loss: 0.9894
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 0.3359
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1544
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0998
Epoch 5/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0758
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0635
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0557
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0504
Epoch 9/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0466
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0434


Because the softmax is integrated into the output layer, the output is a vector of probabilities.

In [9]:
p_nonpreferred = model.predict(X_train)
print(p_nonpreferred [:2])
print("largest value", np.max(p_nonpreferred), "smallest value", np.min(p_nonpreferred))

[[2.18e-03 6.07e-03 9.63e-01 2.84e-02]
 [9.91e-01 8.67e-03 5.42e-06 1.38e-06]]
largest value 0.999998 smallest value 4.9566756e-10


### Preferred <img align="Right" src="./images/C2_W2_softmax_accurate.png"  style=" width:400px; padding: 10px 20px ; ">
Recall from lecture, more stable and accurate results can be obtained if the softmax and loss are combined during training.   This is enabled by the 'preferred' organization shown here.


In the preferred organization the final layer has a linear activation. For historical reasons, the outputs in this form are referred to as *logits*. The loss function has an additional argument: `from_logits = True`. This informs the loss function that the softmax operation should be included in the loss calculation. This allows for an optimized implementation.

In [10]:
preferred_model = Sequential(
    [ 
        Dense(25, activation = 'relu'),
        Dense(15, activation = 'relu'),
        Dense(4, activation = 'linear')   #<-- Note
    ]
)
preferred_model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  #<-- Note
    optimizer=tf.keras.optimizers.Adam(0.001),
)

preferred_model.fit(
    X_train,y_train,
    epochs=10
)
        

Epoch 1/10
63/63 [==============================] - 0s 1ms/step - loss: 0.8609
Epoch 2/10
63/63 [==============================] - 0s 1ms/step - loss: 0.4306
Epoch 3/10
63/63 [==============================] - 0s 1ms/step - loss: 0.2049
Epoch 4/10
63/63 [==============================] - 0s 1ms/step - loss: 0.1085
Epoch 5/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0736
Epoch 6/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0582
Epoch 7/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0497
Epoch 8/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0437
Epoch 9/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0393
Epoch 10/10
63/63 [==============================] - 0s 1ms/step - loss: 0.0350


#### Output Handling
Notice that in the preferred model, the outputs are not probabilities, but can range from large negative numbers to large positive numbers. The output must be sent through a softmax when performing a prediction that expects a probability. 
Let's look at the preferred model outputs:

In [11]:
p_preferred = preferred_model.predict(X_train)
print(f"two example output vectors:\n {p_preferred[:2]}")
print("largest value", np.max(p_preferred), "smallest value", np.min(p_preferred))

two example output vectors:
 [[-3.9  -1.78  3.93 -0.05]
 [ 7.73  2.65 -2.04 -0.9 ]]
largest value 16.324047 smallest value -9.428856


The output predictions are not probabilities!
If the desired output are probabilities, the output should be be processed by a [softmax](https://www.tensorflow.org/api_docs/python/tf/nn/softmax).

In [12]:
sm_preferred = tf.nn.softmax(p_preferred).numpy()
print(f"two example output vectors:\n {sm_preferred[:2]}")
print("largest value", np.max(sm_preferred), "smallest value", np.min(sm_preferred))

two example output vectors:
 [[3.89e-04 3.24e-03 9.78e-01 1.84e-02]
 [9.94e-01 6.22e-03 5.68e-05 1.78e-04]]
largest value 0.9999999 smallest value 1.526002e-11


To select the most likely category, the softmax is not required. One can find the index of the largest output using [np.argmax()](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html).

In [14]:
for i in range(len(p_preferred)):
    print( f"{p_preferred[i]}, category: {np.argmax(p_preferred[i])}")

[-3.9  -1.78  3.93 -0.05], category: 2
[ 7.73  2.65 -2.04 -0.9 ], category: 0
[ 5.56  2.56 -1.51 -1.2 ], category: 0
[-1.37  4.4  -1.24 -1.74], category: 1
[-1.05 -2.5   5.27 -0.7 ], category: 2
[-4.14 -2.15  4.82 -0.84], category: 2
[-2.3   5.23 -1.25 -1.63], category: 1
[10.26  1.45 -1.56 -0.01], category: 0
[-1.44  4.86 -1.46 -1.89], category: 1
[-5.63 -2.27  1.45  7.54], category: 3
[-1.89  2.35  0.04 -0.99], category: 1
[-4.96 -2.74  5.87 -0.93], category: 2
[ 8.58  0.26  0.39 -0.49], category: 0
[-4.99 -3.15  6.96 -1.34], category: 2
[-4.43 -2.25  4.73 -0.2 ], category: 2
[ 8.83  2.21 -1.9  -0.43], category: 0
[-2.69  7.67 -2.3  -2.54], category: 1
[-2.93 -1.2   3.22 -0.78], category: 2
[-5.36e+00  4.88e-04 -2.37e-01  7.80e+00], category: 3
[11.22  1.37 -1.62  0.1 ], category: 0
[11.04  0.75 -0.92  0.1 ], category: 0
[-1.6   6.67 -2.4  -2.41], category: 1
[-2.08  7.81 -2.72 -2.75], category: 1
[-1.67  3.98 -0.87 -1.48], category: 1
[-7.92 -4.62  7.86  1.59], category: 2
[-0.99  4

[-6.33 -3.03  2.18  7.44], category: 3
[-6.75 -3.11  1.58  9.94], category: 3
[-8.38 -4.53  3.05  9.67], category: 3
[-2.74  8.08 -2.49 -2.7 ], category: 1
[-1.82  9.36 -3.83 -3.15], category: 1
[-7.01 -3.52  2.16  9.03], category: 3
[-1.47  6.66 -2.48 -2.4 ], category: 1
[-0.58  5.24 -2.09 -2.17], category: 1
[-8.13 -4.44  3.94  7.53], category: 3
[-5.43 -1.37  0.73  7.63], category: 3
[-7.65 -4.05  2.53  9.52], category: 3
[ 7.38  1.92 -1.44 -0.57], category: 0
[ 7.27  0.64 -0.32 -0.29], category: 0
[-3.5   3.74 -0.97  1.1 ], category: 1
[ 8.26  1.6  -1.39 -0.32], category: 0
[-4.2  -2.2   4.88 -0.82], category: 2
[ 9.93  2.81 -2.57 -0.64], category: 0
[-2.41  8.14 -2.71 -2.8 ], category: 1
[-8.08 -4.35  2.57 10.35], category: 3
[-1.86  4.38 -0.99 -1.53], category: 1
[-3.97 -3.25  6.85 -1.08], category: 2
[-7.47 -3.62  1.58 11.64], category: 3
[-4.91 -1.78  1.5   5.61], category: 3
[-2.75 -1.15  3.22 -0.94], category: 2
[-2.63  7.39 -2.19 -2.45], category: 1
[-5.28 -2.95  6.29 -1.01]

[-1.6   8.13 -3.24 -2.83], category: 1
[-2.57  4.48 -0.97 -0.85], category: 1
[ 8.49  1.7  -1.5  -0.31], category: 0
[ 9.29  2.6  -2.3  -0.61], category: 0
[-7.23 -4.39  8.72 -1.02], category: 2
[-6.14 -1.65  0.48  9.71], category: 3
[-1.59  6.68 -2.41 -2.41], category: 1
[ 6.87  0.54 -0.08 -0.37], category: 0
[-8.17 -4.13  1.78 12.54], category: 3
[-1.09  4.96 -1.71 -1.97], category: 1
[11.34  1.88 -2.1   0.09], category: 0
[ 1.67  3.51 -1.54 -1.76], category: 1
[ 7.13  0.69 -0.36 -0.3 ], category: 0
[-6.48 -2.38  0.92 10.22], category: 3
[ 5.67  2.47 -1.47 -1.12], category: 0
[-5.29 -1.89  1.33  6.74], category: 3
[ 0.77  5.66 -2.69 -2.24], category: 1
[-9.41 -5.17  3.11 11.51], category: 3
[-2.17  7.   -2.21 -2.46], category: 1
[ 6.94  2.12 -1.48 -0.71], category: 0
[ 5.04  0.74 -0.08 -0.57], category: 0
[-5.96 -2.78  2.76  5.15], category: 3
[ 8.07  1.29 -1.09 -0.26], category: 0
[-0.18 -1.64  3.92 -0.76], category: 2
[-0.02  4.95 -2.09 -2.08], category: 1
[-0.44  3.91 -1.39 -1.81]

## SparseCategorialCrossentropy or CategoricalCrossEntropy
Tensorflow has two potential formats for target values and the selection of the loss defines which is expected.
- SparseCategorialCrossentropy: expects the target to be an integer corresponding to the index. For example, if there are 10 potential target values, y would be between 0 and 9. 
- CategoricalCrossEntropy: Expects the target value of an example to be one-hot encoded where the value at the target index is 1 while the other N-1 entries are zero. An example with 10 potential target values, where the target is 2 would be [0,0,1,0,0,0,0,0,0,0].


## Congratulations!
In this lab you 
- Became more familiar with the softmax function and its use in softmax regression and in softmax activations in neural networks. 
- Learned the preferred model construction in Tensorflow:
    - No activation on the final layer (same as linear activation)
    - SparseCategoricalCrossentropy loss function
    - use from_logits=True
- Recognized that unlike ReLU and Sigmoid, the softmax spans multiple outputs.